In [3]:
from sportsreference.nfl.teams import Teams
import pandas as pd
import numpy as np
from scipy import stats
from datetime import datetime

# Pull 2020 NFL Data from sportsreference api

teams = Teams(year= '2020')
teams_df = teams.dataframes
teams_df.sort_values(by=['name'], inplace=True, ascending=True)
teams_df.set_index('name', inplace=True)

# Drop statistics that are not Relevant

teams_df.drop(['first_downs', 'first_downs_from_penalties',  'games_played','losses', 'abbreviation','pass_attempts', 'pass_completions', 'pass_first_downs','plays', 'points_contributed_by_offense','post_season_result', 'rush_attempts', 'rush_first_downs', 'wins'], axis=1, inplace= True)

# Normalize data

for (columnName, columnData) in teams_df.iteritems(): 
    if columnName != 'name':
        teams_df[columnName] = stats.zscore(columnData)

# Invert stats that negatively affect a team

teams_df[['fumbles', 'interceptions', 'penalties', 'percent_drives_with_turnovers', 'points_against', 'turnovers', 'yards_from_penalties']] *= -1
teams_df.head()

defensive_simple_rating_system   fumbles  interceptions  \
name                                                                         
Arizona Cardinals                        0.298534 -0.146889       0.173235   
Atlanta Falcons                          0.239198 -0.482636      -0.354718   
Baltimore Ravens                         1.277577 -0.482636      -0.354718   
Buffalo Bills                            0.179862  0.860350      -0.354718   
Carolina Panthers                        0.387538 -1.154128       0.965164   

                   margin_of_victory  offensive_simple_rating_system  \
name                                                                   
Arizona Cardinals           0.431262                        0.373906   
Atlanta Falcons            -0.175699                       -0.024098   
Baltimore Ravens            1.645186                        0.970911   
Buffalo Bills               1.261842                        1.766918   
Carolina Panthers          -0.527098    

,defensive_simple_rating_system,fumbles,interceptions,margin_of_victory,offensive_simple_rating_system,pass_net_yards_per_attempt,pass_touchdowns,pass_yards,penalties,percent_drives_with_points,...,rush_touchdowns,rush_yards,rush_yards_per_attempt,simple_rating_system,strength_of_schedule,turnovers,win_percentage,yards,yards_from_penalties,yards_per_play
name,,,,,,,,,,,,,,,,,,,,,
Arizona Cardinals,0.298534,0.146889,-0.173235,0.431262,0.373906,0.095371,-0.025220,0.141967,-2.014801,0.039590,...,1.119625,0.903310,0.749012,0.449286,-0.085829,-0.043874,-0.001027,0.825608,-0.693817,0.254467
Atlanta Falcons,0.239198,0.482636,0.354718,-0.175699,-0.024098,0.095371,-0.025220,1.004256,0.598995,0.777262,...,-0.755096,-0.998395,-1.624095,0.120170,1.529776,0.557832,-1.174777,0.304131,0.330181,-0.174109
Baltimore Ravens,1.277577,0.482636,0.354718,1.645186,0.970911,-0.182073,-0.025220,-2.128536,-1.230662,0.857444,...,1.536229,3.152986,2.647497,1.436633,-1.620654,0.557832,0.881633,0.132327,-1.415271,0.683043
Buffalo Bills,0.179862,-0.860350,0.354718,1.261842,1.766918,1.343869,1.473561,1.500024,-1.056409,1.514934,...,-0.130189,-0.483181,-0.437541,1.332701,-0.166609,-0.244443,1.468508,1.209641,-1.260119,1.111618
Carolina Panthers,0.387538,1.154128,-0.965164,-0.527098,-0.596228,0.234093,-1.293419,0.087953,-0.446523,0.055626,...,0.494718,-0.534433,-0.437541,-0.191623,1.772117,-0.043874,-0.878992,-0.308300,0.206060,0.040179


In [ ]:
rank = pd.Series()
rank['defensive_simple_rating_system'] = 5
rank['fumbles'] =0
rank['interceptions'] =0
rank['margin_of_victory'] = 3
rank['offensive_simple_rating_system'] = 5
rank['pass_net_yards_per_attempt'] = .5
rank['pass_touchdowns'] = 1
rank['pass_yards'] =1
rank['penalties'] =1
rank['percent_drives_with_points'] =2
rank['percent_drives_with_turnovers'] = 2
rank['points_against'] =1
rank['rank'] = 0
rank['rush_touchdowns'] = 1
rank['rush_yards'] = 3
rank['rush_yards_per_attempt'] =1
rank['simple_rating_system'] = 7
rank['strength_of_schedule'] = 4
rank['turnovers'] = 3
rank['win_percentage'] = 6
rank['yards'] = 2
rank['yards_from_penalties'] = .5
rank['yards_per_play'] = 2  
sum = rank.sum() 
rank/=sum

In [ ]:
for (columnName, columnData) in rank.iteritems(): 
    teams_df[columnName]*= columnData
    print(rank[columnName])

In [ ]:
teams_df['sum'] = 0.0
for i, row in teams_df.iterrows():
    teams_df.at[i, 'sum'] = row['defensive_simple_rating_system':].sum()
teams_df.sort_values(by=['sum'], inplace=True, ascending=False)
print(teams_df['sum'])

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.kdeplot(teams_df['sum'])



In [ ]:
teams_df['zscores'] = stats.zscore(teams_df['sum'])
teams_df['percentile'] =  1- stats.norm.sf(teams_df['zscores'])
print(teams_df['percentile'])

In [ ]:
def predict (team1, team2):
    t1 = teams_df['percentile'].loc[team1]
    t2 = teams_df['percentile'].loc[team2]
    #return (teams_df['percentile'].loc[team1] - teams_df['percentile'].loc[team2])/2 + .5
    p = 1/(10**(-(t1 - t2))+1)
    return probToMoneyLine(p)

In [ ]:
def probToMoneyLine (prob):
    ml = 0
    prob*=100
    if prob >50:
        ml = -(prob/(100 - prob)) * 100
    elif prob < 50:
        ml = (((100 - prob)/prob) * 100)
    else:
        ml = 100
    return ml

In [ ]:
predict('Tennessee Titans', 'Las Vegas Raiders')

In [ ]:
import json
import requests
import argparse

parser = argparse.ArgumentParser(description='Sample')
parser.add_argument('--api-key', type=str, default='')
args, unknown = parser.parse_known_args()


API_KEY = '58f860df380e5b01f108f9418584b714'

SPORT = 'americanfootball_nfl' # use the sport_key from the /sports endpoint below, or use 'upcoming' to see the next 8 games across all sports

REGION = 'us' # uk | us | eu | au

MARKET = 'h2h' # h2h | spreads | totals

ODDSFORMAT  = 'american'



# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 
#
# Now get a list of live & upcoming games for the sport you want, along with odds for different bookmakers
#
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 

odds_response = requests.get('https://api.the-odds-api.com/v3/odds', params={
    'api_key': API_KEY,
    'sport': SPORT,
    'region': REGION,
    'mkt': MARKET,
    'oddsFormat': ODDSFORMAT,
})

odds_json = json.loads(odds_response.text)
games = []
if not odds_json['success']:
    print(odds_json['msg'])
else:
    print('Number of events:', len(odds_json['data']))
    print(odds_json['data'][0]['commence_time'])
    first = odds_json['data']
    for i, game in enumerate(odds_json['data'], start=0):
        games.append({})
        games[i]['teams'] = game['teams']
        games[i]['home'] = game['home_team']

        for site in game['sites']:
            if site['site_nice'] == 'Caesars':
                games[i]['odds'] = site['odds']['h2h']

    # Check your usage
    print('Remaining requests', odds_response.headers['x-requests-remaining'])
    print('Used requests', odds_response.headers['x-requests-used'])

In [ ]:
print(games)

In [ ]:
for game in games:
    team1 = game['teams'][0]
    team2 = game['teams'][1]
    print(team1, ' vs', team2)
    print('Predicted Line for', team1,'is', predict(team1, team2))
    print('Actual Line for', team1,'is', game['odds'][0] )
